# Notebook 13: A full fledged stereonet

We are now in the position to implement a full fledged stereonet that includes the great and small circles defining its grid.

## Python function 1: View direction

Normally, stereonets are displayed looking straight down, with the primitive equal to the horizontal. However, sometimes it is convenient to look at the stereonet in another orientation. For example, one may want to plot data on the plane of a cross section with the view direction perpendicular to the section, or in a down-plunge projection with the view direction parallel to the fold axis. Function [geogr_to_view](../functions/geogr_to_view.py) enables plotting great and small circles on a stereonet of any view direction, by transforming the poles of rotation from NED coordinates to the view direction coordinates.

## Python function 2: The stereonet

We can now plot a full fledged stereonet in any view direction using the function [stereonet](../functions/stereonet.py). 

## Application:

Let"s plot an equal angle and an equal area stereonet side by side, and let the user interactively choose the view direction using sliders:

In [1]:
# this makes visible our functions folder
import sys, os
sys.path.append(os.path.abspath(os.path.join("..", "functions")))

# import stereonet function
from stereonet import stereonet

# import required libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed

def plot_stereonets(trend_view=0,plunge_view=90,interval=10):
    """
    plot_stereonets plots equal angle and equal area
    stereonets in any view direction

    Input: In degrees
    trend_view = trend of view direction 
    plunge_view = plunge of view direction 
    interval = grid interval

    Returns:
    None
    """
    # angles in radians
    trdv = np.radians(trend_view)
    plgv = np.radians(plunge_view)
    intrad = np.radians(interval)
    
    # figure
    fig, ax = plt.subplots(1,2,figsize=(12,6))

    # plot stereonets
    for i in range(2):
        stereonet(trdv,plgv,intrad,i, ax[i])
        ax[i].set_title(["Equal angle", "Equal area"][i] + " stereonet")
    
    plt.show()

# interact with the function
interact(plot_stereonets, trend_view=(0,360,1), plunge_view=(0,90,1), interval=(2,10,1));

interactive(children=(IntSlider(value=0, description='trend_view', max=360), IntSlider(value=90, description='…

Now let's plot in addition some data:

In [2]:
# import pole_from_plane and plane_from_pole,
# st_coord_line, great_circle, small_circle 
# and geogr_to_view functions
from pole import pole_from_plane, plane_from_pole
from st_coord_line import st_coord_line
from great_circle import great_circle
from small_circle import small_circle
from geogr_to_view import geogr_to_view

def plot_stereonets_and_data(trend_view=0,plunge_view=90,interval=10, lines=[], planes=[], smalls=[]):
    """
    plot_stereonets_and_data plots data on equal angle and equal area
    stereonets in any view direction

    Input: In degrees and RHR convention
    trend_view = trend of view direction 
    plunge_view = plunge of view direction 
    interval = grid interval
    lines = list of lines (trend, plunge)
    planes = list of planes (strike, dip)
    smalls = list of small circles (trend, plunge, apical angle)

    Returns:
    None
    """
    # angles in radians
    trdv = np.radians(trend_view)
    plgv = np.radians(plunge_view)
    intrad = np.radians(interval)
    lines = np.radians(lines)
    planes = np.radians(planes)
    smalls = np.radians(smalls)
    
    # figure
    fig, ax = plt.subplots(1,2,figsize=(12,6))

    # plot stereonets and data
    for i in range(2):
        # stereonets
        stereonet(trdv,plgv,intrad,i, ax[i])
        ax[i].set_title(["Equal angle", "Equal area"][i] + " stereonet")
        # lines
        for line in lines:
            trd, plg = line
            # if not in the default view direction
            if trdv != 0.0 or plgv != np.pi/2:
                trd, plg = geogr_to_view(trd, plg, trdv, plgv)
            xp, yp = st_coord_line(trd, plg, i)
            ax[i].plot(xp, yp, "bo")
        # planes
        for plane in planes:
            stk, dip = plane
            trd, plg = pole_from_plane(stk, dip)
            # if not in the default view direction
            if trdv != 0.0 or plgv != np.pi/2:
                trd, plg = geogr_to_view(stk, dip, trdv, plgv)
            stk, dip = plane_from_pole(trd, plg)
            path = great_circle(stk, dip, i)
            ax[i].plot(path[:,0], path[:,1], "r-")
        # small circles
        for small in smalls:
            trd, plg, apc = small
            # if not in the default view direction
            if trdv != 0.0 or plgv != np.pi/2:
                trd, plg = geogr_to_view(trd, plg, trdv, plgv)
            path1, path2, np1, np2 = small_circle(trd, plg, apc, i)
            ax[i].plot(path1[:np1,0], path1[:np1,1], "b-")
            if np2 > 0:
                ax[i].plot(path2[:np2,0], path2[:np2,1], "b-")

    plt.show()

# data
lines = [[0, 45], [90, 30], [180, 60], [270, 75]]
planes = [[0, 45], [90, 30], [180, 60], [270, 75]]
smalls = [[0, 45, 10], [90, 30, 10], [180, 60, 10], [270, 75, 10]]

# interact with the function
interact(plot_stereonets_and_data, trend_view=(0,360,1), plunge_view=(0,90,1), 
         interval=(2,10,1), lines=fixed(lines), planes=fixed(planes), smalls=fixed(smalls));

interactive(children=(IntSlider(value=0, description='trend_view', max=360), IntSlider(value=90, description='…

Isn't this fantastic 😀 We have made our own stereonet program!